In [1]:
!conda install -c conda-forge folium=0.5.0 --yes 
!pip install shapely

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.0-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   1.00 MB/s
branca-0.3.0-p 100% |################################| Time: 0:00:00  23.55 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  28.39 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  36.82 MB/s
    100% |████████████████████████████████| 1.5MB 327kB/s ta 0:00:01


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import folium
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
!wget -q -O 'london_data.json' http://data.insideairbnb.com/united-kingdom/england/london/2018-08-08/visualisations/neighbourhoods.geojson
!wget -q -O 'san_francisco_data.json' http://data.insideairbnb.com/united-states/ca/san-francisco/2018-08-06/visualisations/neighbourhoods.geojson
!wget -q -O 'sydney_data.json' http://data.insideairbnb.com/australia/nsw/sydney/2018-08-08/visualisations/neighbourhoods.geojson
!wget -q -O 'amsterdam_data.json' http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2018-08-07/visualisations/neighbourhoods.geojson
!wget -q -O 'paris_data.json' http://data.insideairbnb.com/france/ile-de-france/paris/2018-08-13/visualisations/neighbourhoods.geojson

In [4]:
cities = ['Paris', 'London', 'Sydney', 'San Francisco', 'Amsterdam']
geo_data ={}

with open('london_data.json') as json_data:
    geo_data['London'] = json.load(json_data)
    
with open('san_francisco_data.json') as json_data:
    geo_data['San Francisco'] = json.load(json_data)    
    
with open('sydney_data.json') as json_data:
    geo_data['Sydney'] = json.load(json_data)
    
with open('amsterdam_data.json') as json_data:
    geo_data['Amsterdam'] = json.load(json_data)      
    
with open('paris_data.json') as json_data:
    geo_data['Paris'] = json.load(json_data)           
    


In [5]:
from shapely.geometry import shape

column_names = ['City', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

for city in cities:
    for data in geo_data[city]['features']:
        neighborhood_name = data['properties']['neighbourhood']
        
        neighborhood_latlon = shape(data['geometry']).representative_point()._get_coords()[0]
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
        
        neighborhoods = neighborhoods.append({'City':city,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,City,Neighborhood,Latitude,Longitude
0,Paris,Batignolles-Monceau,48.887034,2.306204
1,Paris,Palais-Bourbon,48.854715,2.312969
2,Paris,Buttes-Chaumont,48.887149,2.383096
3,Paris,Opéra,48.877029,2.337891
4,Paris,Entrepôt,48.875907,2.360472


In [6]:
city = 'Paris'

df = neighborhoods[neighborhoods['City'] == city]

latitude = df.iloc[0]['Latitude']
longitude = df.iloc[0]['Longitude']

kclusters = 3
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

mapc = folium.Map(location=[latitude, longitude], zoom_start=10)


for i in range(0,len(geo_data[city]['features'])):
    folium.GeoJson(geo_data[city]['features'][i]['geometry']).add_to(mapc)

for lat, lng, city, neighborhood in zip(df['Latitude'], df['Longitude'], df['City'], df['Neighborhood']):

    label = folium.Popup(neighborhood, parse_html=True)
    
    cluster = 1
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapc)
    
mapc

In [7]:
# The code was removed by Watson Studio for sharing.

In [8]:
def getNearbyVenues(cities, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for city, name, lat, lng in zip(cities, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            city,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City','Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
venues = getNearbyVenues(cities=neighborhoods['City'], names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Batignolles-Monceau
Palais-Bourbon
Buttes-Chaumont
Opéra
Entrepôt
Gobelins
Vaugirard
Reuilly
Louvre
Luxembourg
Élysée
Temple
Ménilmontant
Panthéon
Passy
Observatoire
Popincourt
Bourse
Buttes-Montmartre
Hôtel-de-Ville
Kingston upon Thames
Croydon
Bromley
Hounslow
Ealing
Havering
Hillingdon
Harrow
Brent
Barnet
Enfield
Waltham Forest
Redbridge
Sutton
Lambeth
Southwark
Lewisham
Greenwich
Bexley
Richmond upon Thames
Merton
Wandsworth
Hammersmith and Fulham
Kensington and Chelsea
City of London
Westminster
Camden
Tower Hamlets
Islington
Hackney
Haringey
Newham
Barking and Dagenham
Pittwater
Randwick
Waverley
Warringah
Lane Cove
North Sydney
Bankstown
Canterbury
Strathfield
Auburn
The Hills Shire
Hunters Hill
Sydney
Botany Bay
Hurstville
Fairfield
Blacktown
Holroyd
Ashfield
Marrickville
Sutherland Shire
Rockdale
City Of Kogarah
Parramatta
Ryde
Leichhardt
Hornsby
Ku-Ring-Gai
Willoughby
Liverpool
Penrith
Canada Bay
Camden
Campbelltown
Mosman
Manly
Woollahra
Burwood
Seacliff
Haight Ashbury
Outer

In [10]:
venues.groupby(['City', 'Neighborhood']).count()

Neighborhood Latitude  \
City          Neighborhood                                                    
Amsterdam     Bijlmer-Centrum                                            18   
              Bijlmer-Oost                                               16   
              Bos en Lommer                                              41   
              Buitenveldert - Zuidas                                     45   
              Centrum-Oost                                               48   
              Centrum-West                                              100   
              De Aker - Nieuw Sloten                                     15   
              De Baarsjes - Oud-West                                     79   
              De Pijp - Rivierenbuurt                                    37   
              Gaasperdam - Driemond                                       5   
              Geuzenveld - Slotermeer                                     8   
              IJburg - Zeeburgereiland                                    6   
              Noord-West                                                  2   
              Oostelijk Havengebied - Indische Buurt                      5   
              Osdorp                                                     14   
              Oud-Noord                                                  16   
              Oud-Oost                                                   62   
              Slotervaart                                                14   
              Watergraafsmeer                                             4   
              Westerpark                                                 40   
              Zuid                                                       28   
London        Barking and Dagenham                                        5   
              Barnet                                                      5   
              Bexley                                                     10   
              Brent                                                      16   
              Bromley                                                     4   
              Camden                                                     30   
              City of London                                            100   
              Croydon                                                     2   
              Ealing                                                     21   
              Enfield                                                    49   
              Greenwich                                                   5   
              Hackney                                                    39   
              Hammersmith and Fulham                                     20   
              Haringey                                                   38   
              Harrow                                                      8   
              Havering                                                   15   
              Hillingdon                                                  5   
              Hounslow                                                   11   
              Islington                                                  65   
              Kensington and Chelsea                                     25   
              Kingston upon Thames                                       12   
              Lambeth                                                    18   
              Lewisham                                                   10   
              Merton                                                     16   
              Newham                                                      4   
              Redbridge                                                   3   
              Richmond upon Thames                                        3   
              Southwark                                                  36   
              Sutton     

In [18]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 383 uniques categories.


In [58]:
# one hot encoding
cities_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood and city columns back to dataframe
 
cities_onehot['Neighborhood'] = venues['Neighborhood']
cities_onehot['City'] = venues['City']

columns = list(cities_onehot.columns)
columns.remove('Neighborhood')
columns.remove('City')

fixed_columns = ['City', 'Neighborhood'] + columns
cities_onehot = cities_onehot[list(fixed_columns)]
cities_onehot.head()

,City,Neighborhood,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basque Restaurant,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burgundian Restaurant,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Ch'ti Restaurant,Chaat Place,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,P

In [59]:
cities_onehot.shape


(4620, 384)

In [62]:
cities_grouped = cities_onehot.groupby(['City', 'Neighborhood']).mean().reset_index()
cities_grouped

,City,Neighborhood,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basque Restaurant,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burgundian Restaurant,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Ch'ti Restaurant,Chaat Place,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,P

In [63]:
cities_grouped.shape

(142, 384)

In [75]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

cities_grouped_clustering = cities_grouped.drop(['City','Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cities_grouped_clustering)

# check cluster labels generated for each row in the dataframe


In [74]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1], dtype=int32)